### <font color='darkred'>IA Aplicado a Finanças</font>
### <font color='darkgreen'>Auditoria Contábil </font>
### <font color='darkblue'> Aprendizado de Máquina não Supervisionado DBSCAN</font>

**O DBSCAN (Density-Based Spatial Clustering of Applications with Noise) é um algoritmo popular de clustering baseado em densidade.**

- Em vez de usar a noção tradicional de centróides (como no k-means) para identificar clusters,o DBSCAN identifica regiões densas de pontos de dados no espaço. Essas regiões são separadas por regiões de baixa densidade.


- **Aqui está uma descrição passo a passo de como o DBSCAN funciona:**

> **1- Parâmetros Iniciais:**

- **O algoritmo necessita de dois parâmetros:eps (epsilon) e minPts.**
- eps: Define o raio de uma vizinhança.
- minPts: Número mínimo de pontos necessários para formar uma região densa.


> **2- Processo de Clustering:** 

 **Para cada ponto do conjunto de dados:**

- Se o ponto já foi visitado, vá para o próximo ponto.
- Se não, marque o ponto como visitado.
- Consulte a vizinhança do ponto dentro do raio eps.
- Se há menos pontos na vizinhança do que minPts, marque esse ponto como ruído (ou ponto de borda).
- Caso contrário, inicie um novo cluster e adicione todos os pontos densamente conectados a esse ponto ao cluster.

> **3- Expansão do Cluster:**

**Para cada ponto novo no cluster:**

- Consulte os pontos dentro de sua vizinhança.
- Se um ponto da vizinhança ainda não foi visitado, marque-o como visitado e repita o processo para ele.
- Se o ponto tem uma quantidade suficiente de pontos em sua vizinhança, expanda o cluster para incluí-lo.

> **4- Finalização:**

O processo termina quando todos os pontos foram visitados. No final, todos os pontos estarão ou em um cluster específico ou marcados como ruído.


> **Características do DBSCAN:**

- **Vantagens:**

- Pode  encontrar  clusters  de  formas  arbitrárias,  ao  contrário  do  k-means,  que assume que os clusters são convexos e isotrópicos.
- Não requer que o número de clusters seja especificado antecipadamente.
- É robusto a pontos de ruído.


- **Desvantagens:**

- Não se sai bem quando os clusters têm variações significativas de densidade.
- A  escolha  dos  parâmetros  eps  e  minPts  pode  ser  desafiadora  e  pode  exigir experimentação.

ODBSCAN é uma técnica poderosa, especialmente quando os conjuntos de dados têm clusters  de  formas  complexas  e  não  uniformes e  quando  a  presença  de  ruído  é  uma preocupação.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

https://optuna.org/

### <font color='darkred'>Carregando e Instalando Pacotes</font>

In [2]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [5]:
# pacote python para otimização de hiperparâmetros
#!pip install optuna  

     -------------------------------------- 362.8/362.8 kB 3.8 MB/s eta 0:00:00
     -------------------------------------- 233.2/233.2 kB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 78.6/78.6 kB 4.6 MB/s eta 0:00:00


In [6]:
# Imports
import optuna #otimização dos hiperparametros
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN #pacote cluster e dentro o pacote DBSCAN

#pacote preprocessing e dentro a função StandardScaler
# Que permite criar um padronizador dos dados, importante no trabalho com tecnicas de clusterização.
# Clusterização - devemos deixar os dados na mesma escala!
from sklearn.preprocessing import StandardScaler 

# medida de avaliação de algoritmo de clusterização
# Como não temos valores de saídas nos dados, não temos como usar metricas de aprendizado supervisionado como acurácia, precisão 
from sklearn.metrics import silhouette_score 
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Michelle Bouhid" --iversions

Author: Michelle Bouhid

optuna  : 4.0.0
platform: 1.0.8
pandas  : 1.5.3
sklearn : 1.3.0
numpy   : 1.23.5



### <font color='darkred'>Carregando Dados</font>

In [8]:
# Carrega os dados
df = pd.read_csv('transacoes_contabeis.csv')

In [9]:
# Converte a data da transação para uma representação numérica - 
# Não vamos isar data como data, vamos usar como categoria.
# Nesse projeto não estamos trabalhando com séries temporais, não precisamos colocar data como indice
# Queremos agrupar os dados por similaridade, não precisamos de ordem cronologica como série temporais
# Como a data tem traço - computador interpreta como variável categorica, texto que não pode ser usado nesse caso.  
# Temos mtas datas repetidas se comportando como categoria.
# Converte em datetime e depois converte pra inteiro no formato do Numpy pq não pode usar como texto.

df['Data_Transacao'] = pd.to_datetime(df['Data_Transacao']).astype(np.int64)

In [10]:
# Seleciona as colunas que vamos usar para identificar possíveis atividades de smurfing
# Salva as colunas na variavel atributos e chama de X o df [atributos]
atributos = ['ID_Conta', 'Data_Transacao', 'Valor']
X = df[atributos]

**Por que Normalizamos Atributos?**

A normalização dos atributos é uma etapa crucial em muitas técnicas de análise de dados, incluindo a clusterização. No contexto do algoritmo DBSCAN (Density-Based Spatial Clustering of Applications  with  Noise)  e  de  muitos  outros  algoritmos  de  aprendizado  de  máquina,  a normalização é especialmente importante por diversas razões:

- **Escala dos Atributos:** DBSCAN funciona com base na noção de densidade e distância. Se os atributos não estiverem na mesma escala, um atributo com uma variação maior dominará os resultados, tornando a medida de distância efetivamente baseada nesse atributo.


- **Sensibilidade à Distância:** O DBSCAN classifica os pontos como centrais, de fronteira ou de ruído com base na densidade em um raio especificado (eps). Se os atributos não estiverem normalizados, este raio pode não ser representativo para todos os atributos, levando a clusters distorcidos.


- **Melhor Interpretação:** Ao trabalhar com dados em escalas diferentes, pode ser difícil visualizar e interpretar os clusters. A normalização ajuda a colocar todos os atributos em uma escala similar, tornando mais fácil a visualização e interpretação dos clusters.


- **Desempenho do Algoritmo:** Alguns algoritmos podem convergir mais rápido quando os dados são normalizados, pois oproblema se torna mais "uniforme". No caso do DBSCAN, a normalização pode resultar em uma melhor identificação dos clusters e pontos de ruído.


- **Uniformidade  em Diferentes Domínios:**  Ao  lidar  com  atributos  que  representam diferentes  domínios  (por  exemplo,  peso  em  quilogramas  e  altura  em  centímetros),  a normalização garante que cada domínio contribua de maneira equitativa para a análise.


- **Prevenção de Distorções:** Sem normalização, clusters podem ser formados com base apenas nas magnitudes dos atributos, em vez de suas relações, o que pode não ser desejado em muitos cenários.

É importante mencionar que, embora a normalização seja benéfica em muitos cenários, nem sempre é apropriada. Em alguns contextos, as escalas originais dos atributos são cruciais para a análise e a normalização pode obscurecer padrões importantes. Portanto, sempre é crucial entender o problema e os dados com os quais você está trabalhando.

In [11]:
# Normalizar os recursos para que eles tenham uma média de 0 e um desvio padrão de 1
# Cada algoritmo tem premissas e suposições, o DBSCAN precisa que os dados estejam na mesma escala.
# cria o objeto scaler, que é uma instancia da classe StandardScaler
# O objeto scaler, tem metodos e atributos, vamos chamar o "fit"
# fit faz o treinamento do X e transforma o X.
# Aplicando um truque matematico que modifica os dados mas não modifica a informação
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
# Definindo a função de otimização que será utilizada pelo optuna
def optimize(trial):

    # Define um valor para 'eps' no intervalo de 0.1 a 1.0 usando o objeto `trial`
    # Podemos otimizar mais parametros se tiver tempo, em aula so otimizamos eps e min samples
    # usamos valores um pouco abaixo e acima do valor padrão sugerido na documentação
    # suggest_float pq o eps é float
    eps = trial.suggest_float('eps', 0.1, 1.0)

    # Define um valor inteiro para 'min_samples' no intervalo de 2 a 10 usando o objeto `trial`
    # suggest_int pq o min_samples é inteiro
    min_samples = trial.suggest_int('min_samples', 2, 10)

    # Instancia o algoritmo DBSCAN com os parâmetros sugeridos anteriormente
    # chama o modelo DBSCAN e passa os hiperparametros trabalhados acima
    dbscan = DBSCAN(eps = eps, min_samples = min_samples)

    # Aplica o algoritmo DBSCAN no conjunto de dados X e retorna as etiquetas para cada ponto de dado
    # Labels = clusters 
    labels = dbscan.fit_predict(X)
    
    # Usamos o coeficiente silhouette para avaliar a qualidade dos clusters. 
    # O coeficiente silhouette varia de -1 a 1. 
    # Valores Próximos de 1: Indicam que os pontos são muito semelhantes aos outros pontos do cluster e diferentes dos pontos de outros clusters.
    # Valores Próximos de 0: Indicam que os pontos estão próximos da fronteira de decisão entre dois clusters.
    # Valores Próximos de -1: Indicam que os pontos foram atribuídos ao cluster errado. (anomalia!)    
    # Como o coeficiente silhouette é indefinido para um único cluster, verificamos se temos mais de um cluster antes de calcular.
    # Só faz sentido calcular o coeficientes se tiver mais de 1 clusters para comparação
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0) #verifica o numero de cluster
    if n_clusters > 1: #calcula se for maior que 1 cluster
        score = silhouette_score(X, labels)
    else:
        score = -1 # se não for maior que 1 já caracteriza em -1 (modelo ruim)
    return score

In [13]:
# Otimizando os hiperparâmetros
# Cria o objeto study na direção maximize
# A partir do objeto study chama a função optimize e dentro dessa função chama a função criada acima
# Que tb se chama optimize!! (didaticamente pra chamar atenção, pode ser qq função :)
# vai fazer o estudo por 100 tentativas
study = optuna.create_study(direction = 'maximize')
study.optimize(optimize, n_trials = 100) #pode colocar mais trials se quiser mais combinações de otimização

[I 2024-11-02 09:29:36,714] A new study created in memory with name: no-name-97c6898f-494c-4cfd-97ea-0909a6e2e762
[I 2024-11-02 09:29:36,798] Trial 0 finished with value: 0.1689376235248055 and parameters: {'eps': 0.11679733787122164, 'min_samples': 5}. Best is trial 0 with value: 0.1689376235248055.
[I 2024-11-02 09:29:36,819] Trial 1 finished with value: -1.0 and parameters: {'eps': 0.9444631491604687, 'min_samples': 7}. Best is trial 0 with value: 0.1689376235248055.
[I 2024-11-02 09:29:36,879] Trial 2 finished with value: 0.2541009050992981 and parameters: {'eps': 0.25343319191156033, 'min_samples': 3}. Best is trial 2 with value: 0.2541009050992981.
[I 2024-11-02 09:29:36,931] Trial 3 finished with value: 0.12842818750789525 and parameters: {'eps': 0.17647049212324384, 'min_samples': 3}. Best is trial 2 with value: 0.2541009050992981.
[I 2024-11-02 09:29:36,945] Trial 4 finished with value: -1.0 and parameters: {'eps': 0.4876865969964763, 'min_samples': 4}. Best is trial 2 with va

[I 2024-11-02 09:29:39,007] Trial 44 finished with value: 0.16864360504156217 and parameters: {'eps': 0.20360348751804835, 'min_samples': 7}. Best is trial 42 with value: 0.3469100427946081.
[I 2024-11-02 09:29:39,076] Trial 45 finished with value: -0.46213138013499094 and parameters: {'eps': 0.33411473028593547, 'min_samples': 2}. Best is trial 42 with value: 0.3469100427946081.
[I 2024-11-02 09:29:39,140] Trial 46 finished with value: 0.31734427174890295 and parameters: {'eps': 0.26824127548847465, 'min_samples': 2}. Best is trial 42 with value: 0.3469100427946081.
[I 2024-11-02 09:29:39,209] Trial 47 finished with value: 0.1277692590517116 and parameters: {'eps': 0.29237642789993595, 'min_samples': 2}. Best is trial 42 with value: 0.3469100427946081.
[I 2024-11-02 09:29:39,251] Trial 48 finished with value: -1.0 and parameters: {'eps': 0.9967812208690905, 'min_samples': 9}. Best is trial 42 with value: 0.3469100427946081.
[I 2024-11-02 09:29:39,278] Trial 49 finished with value: -1.

[I 2024-11-02 09:29:41,635] Trial 88 finished with value: 0.1897904338563815 and parameters: {'eps': 0.28642523395910197, 'min_samples': 2}. Best is trial 61 with value: 0.36172766840283227.
[I 2024-11-02 09:29:41,698] Trial 89 finished with value: 0.26520258771430916 and parameters: {'eps': 0.26405180545706924, 'min_samples': 3}. Best is trial 61 with value: 0.36172766840283227.
[I 2024-11-02 09:29:41,759] Trial 90 finished with value: 0.16415002608649573 and parameters: {'eps': 0.135761197423663, 'min_samples': 7}. Best is trial 61 with value: 0.36172766840283227.
[I 2024-11-02 09:29:41,822] Trial 91 finished with value: 0.35793526188362884 and parameters: {'eps': 0.23307740543790753, 'min_samples': 2}. Best is trial 61 with value: 0.36172766840283227.
[I 2024-11-02 09:29:41,885] Trial 92 finished with value: 0.34889604377079503 and parameters: {'eps': 0.24550496278341757, 'min_samples': 2}. Best is trial 61 with value: 0.36172766840283227.
[I 2024-11-02 09:29:41,955] Trial 93 finish

### Coeficiente de Avaliação da Performance do Modelo

O  coeficiente  silhouette  é  uma  métrica  usada  para  calcular  a eficiênciade  um agrupamento, ou seja, o quão bem cada objeto está agrupado com outros objetos em um cluster. Ele pode ser aplicado a vários algoritmos de clusterização, incluindo o DBSCAN.

Para um único ponto de dados, o coeficiente silhouette é calculado da seguinte maneira:

- a: É a distância média do ponto para os outros pontos no mesmo cluster. Quanto menor for este valor, melhor.

- b: É a menor distância média do ponto para os pontos em um cluster diferente, do  qual  o  ponto  não  faz  parte.  Idealmente,  este  valor  será  grande  se  o agrupamento for bom.

Para obter o coeficiente silhouette para todo o conjunto de dados, calcula-se a média do silhouette de todos os pontos.

A  métrica  silhouette_score  no  contexto  do  DBSCAN  (ou  qualquer  algoritmo  de clusterização) fornece insights valiosos:

- Valores Próximos de 1: Indicam que os pontos são muito semelhantes aos outros pontos do cluster e diferentes dos pontos de outros clusters.

- Valores Próximos de 0: Indicam que os pontos estão próximos da fronteira de decisão entre dois clusters.

- Valores Próximos de -1: Indicam que os pontos foram atribuídos ao cluster errado.

A  avaliação  com  o  coeficiente  silhouette  pode  ser  particularmente  útil  ao  ajustar hiperparâmetros do DBSCAN (como eps e min_samples) porque fornece uma métrica objetiva para a qualidade dos clusters. No entanto, é importante considerar que, para o DBSCAN, pode haver pontos considerados "ruído" que não pertencem a nenhum cluster. Esses pontos não são diretamente considerados no cálculo do coeficiente silhouette.

Assim, é válido observar que, enquanto o coeficiente silhouette fornece uma métrica objetiva, a melhor configuração para um algoritmo de clusterização pode também depender do contexto e do objetivo específico da análise.


In [14]:
# Obter os melhores hiperparâmetros
best_params = study.best_params

In [15]:
# Cria o DBSCAN com os melhores hiperparâmetros
dbscan = DBSCAN(eps = best_params['eps'], min_samples = best_params['min_samples'])

In [16]:
# Treina o DBSCAN com os melhores hiperparâmetros
# Cria a coluna Cluster no nosso df
df['Cluster'] = dbscan.fit_predict(X)

In [17]:
# Encontraremos agora os clusters que têm um número de transações acima de um determinado limiar.
# Nesse caso, estamos usando um limiar de 5, mas você pode ajustá-lo conforme necessário.
# Regra de auditoria, encontrar cluster com mais de 5 elementos de dados - pouco restritiva
# escolha menos restritiva pra passar pro especialista em contabilidade fazer o filtro tecnico.
# Pela a coluna cluster e faz uma contagem, com base nessa contagem palica um filtro > limiar e busca o indice dela
limiar = 5
clusters_suspeitos = df['Cluster'].value_counts()[df['Cluster'].value_counts() > limiar].index

In [18]:
# Extraindo as transações nos clusters suspeitos
# filtro, a coluna CLuster esta dentro desse objeto "cluster suspeitos"
transacoes_suspeitas = df[df['Cluster'].isin(clusters_suspeitos)]

In [19]:
# Filtrando o dataframe com base no valor da coluna 'Cluster'
# Filtro pela regra do DBSCAN, (-1 == anomalia)
filtrado = transacoes_suspeitas[transacoes_suspeitas['Cluster'] == -1]

In [20]:
# Na implementação DBSCAN do scikit-learn, o rótulo de cluster -1 significa que a amostra foi considerada como ruído.

# O DBSCAN, que significa "Density-Based Spatial Clustering of Applications with Noise", é um algoritmo de agrupamento que cria clusters de 
# regiões de alta densidade no espaço de recursos e identifica amostras individuais que estão em áreas de baixa densidade como ruído.

# Divide os dados em grupos, com base em distancia matemática, se algum dado não couber em nenhum grupo, identifica como anomalia.

# Usa o Optuna, um pacote Python para otimizar hiperparametros e o DBSCAN que é um algoritmo de aprendizado de máquina não supervisionado

# Portanto, em nosso contexto, se uma transação é rotulada com o cluster -1, isso significa que ela não foi incluída em nenhum dos 
# clusters densos de transações formados pelo DBSCAN e foi considerada "ruído". Isso poderia potencialmente indicar uma transação incomum ou anômala, 
# dependendo do contexto específico e dos parâmetros do algoritmo.

# Selecionando as colunas desejadas
resultado = filtrado[['ID_Conta', 'Valor', 'Cluster']]

In [21]:
# Salva o resultado em disco
resultado.to_csv('transacoes_suspeitas.csv', index = False)


In [22]:
# Fim
# o arquivo foi gerado na mesma pasta no Jupyter Notebook